In [40]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [41]:
import pandas as pd
df = pd.read_csv('/content/gdrive/My Drive/Mestrado/second_level_preprocessed.csv')

In [42]:
df

,Unnamed: 0,Unnamed: 0.1,text,labels,text_processed
0,0,0,Hello and hope you are doing well.Understand y...,"['emotional-support', 'informational-support',...",Hello and hope you are doing well.Understand y...
1,1,1,It would be hard to see any significant abnorm...,"['informational-support', 'sharing-experiences...",It would be hard to see any significant abnorm...
2,2,2,"Hi James, I, too, go to bed by 10:00, then wa...","['informational-support', 'sharing-experiences...","Hi James, I, tsurprised, go to bed by <TIME>, ..."
3,3,3,"Hi, I am suffering from the same sleep problem...",['sharing-experiences'],"Hi, am suffering from the same sleep problems...."
4,4,4,"me too, it feels like its morning after 1hour","['sharing-experiences', 'esteem-support']","me tsurprised, it feels like its <TIME> after ..."
...,...,...,...,...,...
448,448,448,I just had a similar experience of feeling a d...,['sharing-experiences'],just had similar eplayfulerience of feeling dr...
449,449,449,Thank you for posting. I saw starting to think...,['sharing-experiences'],Thank you for posting. saw starting to think w...
450,450,450,During an induced 4 week coma following a hear...,['sharing-experiences'],During <DATE> <DISEASE> following heart operat...
451,451,451,I have been doing his for years. especially at...,['sharing-experiences'],have been doing his for <DATE>. especially at ...


In [43]:
def get_tail_label(df):
    """
    Give tail label colums of the given target dataframe
    
    args
    df: pandas.DataFrame, target label df whose tail label has to identified
    
    return
    tail_label: list, a list containing column name of all the tail label
    """
    columns = df.columns
    n = len(columns)
    irpl = np.zeros(n)
    for column in range(n):
        irpl[column] = df[columns[column]].value_counts()[1]
    irpl = max(irpl)/irpl
    mir = np.average(irpl)
    tail_label = []
    for i in range(n):
        if irpl[i] > mir:
            tail_label.append(columns[i])
    return tail_label

def get_index(df):
  """
  give the index of all tail_label rows
  args
  df: pandas.DataFrame, target label df from which index for tail label has to identified
    
  return
  index: list, a list containing index number of all the tail label
  """
  tail_labels = get_tail_label(df)
  index = set()
  for tail_label in tail_labels:
    sub_index = set(df[df[tail_label]==1].index)
    index = index.union(sub_index)
  return list(index)

def get_minority_instace(X, y):
    """
    Give minority dataframe containing all the tail labels
    
    args
    X: pandas.DataFrame, the feature vector dataframe
    y: pandas.DataFrame, the target vector dataframe
    
    return
    X_sub: pandas.DataFrame, the feature vector minority dataframe
    y_sub: pandas.DataFrame, the target vector minority dataframe
    """
    index = get_index(y)
    X_sub = X[X.index.isin(index)].reset_index(drop = True)
    y_sub = y[y.index.isin(index)].reset_index(drop = True)
    return X_sub, y_sub

def nearest_neighbour(X, neigh=5):
    """
    Give index of 5 nearest neighbor of all the instance
    
    args
    X: np.array, array whose nearest neighbor has to find
    
    return
    indices: list of list, index of 5 NN of each element in X
    """
    nbs=NearestNeighbors(n_neighbors=5,metric='euclidean',algorithm='kd_tree').fit(X)
    euclidean,indices= nbs.kneighbors(X)
    return indices



def MLSMOTE(X, y, n_sample = 100, neigh=5):
    """
    Give the augmented data using MLSMOTE algorithm
    
    args
    X: pandas.DataFrame, input vector DataFrame
    y: pandas.DataFrame, feature vector dataframe
    n_sample: int, number of newly generated sample
    
    return
    new_X: pandas.DataFrame, augmented feature vector data
    target: pandas.DataFrame, augmented target vector data
    """
    indices2 = nearest_neighbour(X, neigh=5)
    n = len(indices2)
    new_X = np.zeros((n_sample, X.shape[1]))
    target = np.zeros((n_sample, y.shape[1]))
    for i in range(n_sample):
        reference = random.randint(0, n-1)
        neighbor = random.choice(indices2[reference, 1:])
        all_point = indices2[reference]
        nn_df = y[y.index.isin(all_point)]
        ser = nn_df.sum(axis = 0, skipna = True)
        target[i] = np.array([1 if val > 0 else 0 for val in ser])
        ratio = random.random()
        gap = X.loc[reference,:] - X.loc[neighbor,:]
        new_X[i] = np.array(X.loc[reference,:] + ratio * gap)
    new_X = pd.DataFrame(new_X, columns=X.columns)
    target = pd.DataFrame(target, columns=y.columns)
    return new_X, target

In [44]:
X = df['text_processed']


In [45]:
from ast import literal_eval
df['labels'] = df['labels'].apply(literal_eval)

In [46]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y=mlb.fit_transform(df['labels'])

In [47]:
! pip install iterative-stratification

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
! pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [74]:
def vectorize_sentences(X):

    new_X = []
    for instance in X:
      sentences = sent_tokenize(instance)
      current_embedding = embed(sentences).numpy()
      new_X.append(current_embedding)
    
    return np.array(new_X)

In [49]:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer


#using the count vectorizer
count = CountVectorizer()
word_count=count.fit_transform(df['text_processed'])

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count)
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=count.get_feature_names(),columns=["idf_weights"])


#inverse document frequency
df_idf.sort_values(by=['idf_weights'])

#tfidf
tf_idf_vector=tfidf_transformer.transform(word_count)
feature_names = count.get_feature_names()

first_document_vector=tf_idf_vector[1]
df_tfifd= pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])

df_tfifd.sort_values(by=["tfidf"],ascending=True)

temp = df_tfifd.to_dict()

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stopwords_nltlk = list(set(stopwords.words('english')))

ultimate = []
for word in stopwords_nltlk:
  if word in temp['tfidf'].keys():
    if temp['tfidf'][word] == 0 :#and word not in do_not_remove:
      ultimate.append(word)

#### add ultimate as stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [51]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize 
import numpy as np

def average_sentences(X):

    new_X = []
    for instance in X:
      sentences = sent_tokenize(instance)
      current_embedding = embed(sentences).numpy()
      '''
      diff = number -len(current_embedding)
      if diff > 0:
          padding = np.zeros((diff, 512))
          current_embedding = np.concatenate((padding, current_embedding), axis=0)
      '''
      new_X.append(np.mean(current_embedding, axis=0))
      
    
    return np.array(new_X)


from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [52]:
########### paraphrase ##############
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [53]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
tokenizer = AutoTokenizer.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

device  cpu


In [54]:
def paraphrase(context):

  text = "" + context + " </s>"
  encoding = tokenizer.encode_plus(text,max_length =128, padding=True, return_tensors="pt")
  input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
  model.eval()
  diverse_beam_outputs = model.generate(
      input_ids=input_ids,attention_mask=attention_mask,
      max_length=128,
      early_stopping=True,
      num_beams=10,
      num_beam_groups = 5,
      num_return_sequences=2,
      diversity_penalty = 0.70
  )

  ans = []
  for beam_output in diverse_beam_outputs:
      sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      ans.append(sent[len("'paraphrasedoutput:") : ])
  return ans

In [55]:
from nltk.tokenize import sent_tokenize, word_tokenize 
import nltk
import numpy as np
nltk.download('punkt')

def paraphrase_sample(text):
  n = len(sent_tokenize(text))

  result = []
  smallest = 1000

  for sentence in sent_tokenize(text):
    sentence_augments = paraphrase(sentence)

   
    smallest = min(smallest, len(sentence_augments))
    result.append(sentence_augments[0: smallest])


  result = np.array(result).T
  ans = []
  for i in range(len(result)):
    ans.append(' '.join(result[i]))
    #result[i] = ' '.join(result[i])
      
  return ans

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [56]:
def aug(X_train, y_train, index):
  
  X_new = []
  y_new= []

  for X, y in zip(X_train[index], y_train[index]):
    X_new.append(paraphrase_sample(X))
    y_new.append(y)

  return X_new, y_new


In [57]:
def do(X_train, X_a):
 oop = X_train.tolist()
 for a in X_a:
   print(a[0])
   oop.append(a[0])
 return oop

def do_y(X_train, X_a):
 oop = X_train.tolist()
 for a in X_a:
   
   oop.append(a)
 return oop

In [91]:
def vectorize_sentences(X):
  a = []
  for x in X:
    print(x)
    embeded_tweets = embed(x).numpy()
    a.append(embeded_tweets)
  return a

In [134]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from imblearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier ### does not take into account label correlations
from sklearn.svm import SVC
from sklearn.metrics import classification_report, multilabel_confusion_matrix, accuracy_score, hamming_loss, jaccard_score

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

#from sklearn.multioutput import ClassifierChain

from sklearn.utils import compute_class_weight
from skmultilearn.problem_transform import BinaryRelevance
from keras.wrappers.scikit_learn import KerasClassifier

from skmultilearn.problem_transform import ClassifierChain, LabelPowerset
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from skmultilearn.cluster import NetworkXLabelGraphClusterer
from skmultilearn.cluster import LabelCooccurrenceGraphBuilder
from skmultilearn.ensemble import LabelSpacePartitioningClassifier

msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)


# linear is nice 0.47
# poly 0.37
# rbf 0.34
# sigmoid 0.43

#from skmultilearn.ext import Keras

from skmultilearn.adapt import MLkNN
from sklearn.model_selection import GridSearchCV


################ run the pipeline #####################

#### RF E LR
for train_index, test_index in msss.split(np.array(X), np.array(y)):
      import warnings
      warnings.simplefilter(action='ignore', category=FutureWarning)
      ####### performing the different splits here #########
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]

      from sklearn.feature_extraction.text import TfidfVectorizer
      

      X_train = embed(X_train)
      print(X_train.shape)

      df_y =  pd.DataFrame(y_train, columns=mlb.classes_)
      df_X = pd.DataFrame(data = X_train)

      X_sub, y_sub = get_minority_instace(df_X, df_y)   
      X_res, y_res =MLSMOTE(X_sub, y_sub, X_sub.shape[0])  

      X_train = np.concatenate((X_train, X_res.values))
      y_train = np.concatenate((y_train, y_res.values))

     
      #X_train_augmented = pd.concat([df_X, X_res])
      #y_train_augmented = pd.concat([df_y, y_res])

    
      X_test=  embed(X_test)
      # X_test = vectorizer.transform(X_test)#average_sentences(X_test) #vectorizer.transform(X_test)
      from sklearn.linear_model import LogisticRegression
      from sklearn.naive_bayes import GaussianNB

      clf = ClassifierChain(GaussianNB(var_smoothing=1e-3))
      #clf = BinaryRelevance(SVC(C= 1, kernel="linear", class_weight = "balanced"))
      clf.fit(X_train, y_train)

      y_pred = clf.predict(X_test)
      
      from sklearn.metrics import classification_report, jaccard_score, hamming_loss
      print(classification_report(y_test, y_pred))
      print(jaccard_score(y_test, y_pred, average= "weighted"))
      print(hamming_loss(y_test, y_pred))


(361, 512)
              precision    recall  f1-score   support

           0       0.25      0.07      0.11        15
           1       0.00      0.00      0.00         7
           2       0.77      0.55      0.64        42
           3       0.40      0.10      0.16        20
           4       0.00      0.00      0.00         7
           5       1.00      0.27      0.42        26
           6       0.72      0.45      0.55        40

   micro avg       0.72      0.32      0.45       157
   macro avg       0.45      0.20      0.27       157
weighted avg       0.63      0.32      0.41       157
 samples avg       0.48      0.37      0.39       157

0.28411400266606285
0.1956521739130435


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [127]:
X_res.values.shape, X_train.shape

((30, 512), TensorShape([361, 512]))

In [129]:
np.concatenate((X_res, X_train)).shape

(391, 512)